# Data Privacy and Security in Databricks

This notebook demonstrates data privacy and security features in Databricks Unity Catalog.

## Topics:
1. **RBAC** - Role-Based Access Control
2. **Views** - Dynamic, Restricted, and Materialized
3. **Data Hashing** - Irreversible anonymization
4. **Data Masking** - Format-preserving obfuscation
5. **Row Filtering** - Scope access by attributes
6. **Tokenization** - Reversible token replacement
7. **ABAC** - Attribute-Based Access Control
8. **Encryption** - Protect data at rest and in transit

## Configuration

Set your demo preferences:

In [0]:
# Demo Configuration
USE_TEMP_TABLES = True  # Recommended for demos - auto-cleanup on session end
CREATE_CATALOG_SCHEMA = False  # Recommended for demos - auto-cleanup on session end. If disabled and the catalog and schema doesn't exist, code will panic.

# Catalog and schema names
CATALOG = "main"
HR_SCHEMA = "hr"
CUSTOMERS_SCHEMA = "customers"
RETAIL_SCHEMA = "retail"
GOVERNANCE_SCHEMA = "governance"

print(f"✓ Configuration loaded")
print(f"  → Temporary tables: {USE_TEMP_TABLES}")
print(f"  → Catalog: {CATALOG}")

In [0]:
# Import required libraries and define helper functions
from pyspark.sql.functions import *
from pyspark.sql.types import *

def get_table_type():
    return "TEMPORARY" if USE_TEMP_TABLES else ""

def func_name(name):
    return name if USE_TEMP_TABLES else f"{CATALOG}.{GOVERNANCE_SCHEMA}.{name}"

def view_name(schema, name):
    return name if USE_TEMP_TABLES else f"{CATALOG}.{schema}.{name}"

def table_name(schema, name):
    return name if USE_TEMP_TABLES else f"{CATALOG}.{schema}.{name}"

print("✓ Libraries and helpers loaded")

In [0]:
# ============================================================================
# ENVIRONMENT SETUP
# ============================================================================
# This notebook is called via %run from the main demo notebook
# Configuration is passed from the calling notebook

from pyspark.sql.functions import *
from pyspark.sql.types import *

print("Setting up demo environment...")
print(f"→ Mode: {'Temporary tables' if USE_TEMP_TABLES else 'Permanent tables'}")

# Only create schemas if using permanent tables
if CREATE_CATALOG_SCHEMA and not USE_TEMP_TABLES:
    for schema in [HR_SCHEMA, CUSTOMERS_SCHEMA, RETAIL_SCHEMA, GOVERNANCE_SCHEMA]:
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{schema}")
    print("✓ Schemas created")

# Helper function
def get_create_stmt(table_name, schema, columns):
    if USE_TEMP_TABLES:
        return f"CREATE OR REPLACE TEMP VIEW {table_name} AS SELECT {columns}"
    else:
        return f"CREATE OR REPLACE TABLE {CATALOG}.{schema}.{table_name} ({columns})"

# HR employee_info table or temp view
if USE_TEMP_TABLES:
    spark.sql(f"""
        CREATE OR REPLACE TEMP VIEW employee_info AS
        SELECT * FROM VALUES
            (1, 'David Wells', 100000.00, '123-45-6789'),
            (2, 'Chris Moon', 120000.00, '234-56-7890'),
            (3, 'Jane Doe', 95000.00, '345-67-8901'),
            (4, 'John Smith', 110000.00, '456-78-9012')
        AS t(id, name, salary, ssn)
    """)
else:
    spark.sql(f"""
        CREATE OR REPLACE TABLE {CATALOG}.{HR_SCHEMA}.employee_info (
            id INT, name STRING, salary DECIMAL(10, 2), ssn STRING)
    """)
    spark.sql(f"""
        INSERT INTO {CATALOG}.{HR_SCHEMA}.employee_info VALUES
            (1, 'David Wells', 100000.00, '123-45-6789'),
            (2, 'Chris Moon', 120000.00, '234-56-7890'),
            (3, 'Jane Doe', 95000.00, '345-67-8901'),
            (4, 'John Smith', 110000.00, '456-78-9012')
    """)

# Customers customer_info table or temp view
if USE_TEMP_TABLES:
    spark.sql(f"""
        CREATE OR REPLACE TEMP VIEW customer_info AS
        SELECT * FROM VALUES
            (1, 'david.wells@databricks.com', 'David Wells', '2025-01-01'),
            (2, 'chris.moon@databricks.com', 'Chris Moon', '2025-02-01'),
            (3, 'jane.doe@example.com', 'Jane Doe', '2025-03-15'),
            (4, 'john.smith@example.com', 'John Smith', '2025-04-20')
        AS t(id, email, name, created_at)
    """)
else:
    spark.sql(f"""
        CREATE OR REPLACE TABLE {CATALOG}.{CUSTOMERS_SCHEMA}.customer_info (
            id INT, email STRING, name STRING, created_at DATE)
    """)
    spark.sql(f"""
        INSERT INTO {CATALOG}.{CUSTOMERS_SCHEMA}.customer_info VALUES
            (1, 'david.wells@databricks.com', 'David Wells', '2025-01-01'),
            (2, 'chris.moon@databricks.com', 'Chris Moon', '2025-02-01'),
            (3, 'jane.doe@example.com', 'Jane Doe', '2025-03-15'),
            (4, 'john.smith@example.com', 'John Smith', '2025-04-20')
    """)

# Retail customers table or temp view
if USE_TEMP_TABLES:
    spark.sql(f"""
        CREATE OR REPLACE TEMP VIEW customers AS
        SELECT * FROM VALUES
            (1, '123-45-6789', 'Alice Smith', 'US'),
            (2, '234-56-7890', 'Maria Silva', 'EU'),
            (3, '456-78-9012', 'Akira Tanaka', 'APAC'),
            (4, '567-89-0123', 'Bob Johnson', 'US'),
            (5, '678-90-1234', 'Emma Brown', 'EU')
        AS t(id, ssn, name, region)
    """)
else:
    spark.sql(f"""
        CREATE OR REPLACE TABLE {CATALOG}.{RETAIL_SCHEMA}.customers (
            id INT, ssn STRING, name STRING, region STRING)
    """)
    spark.sql(f"""
        INSERT INTO {CATALOG}.{RETAIL_SCHEMA}.customers VALUES
            (1, '123-45-6789', 'Alice Smith', 'US'),
            (2, '234-56-7890', 'Maria Silva', 'EU'),
            (3, '456-78-9012', 'Akira Tanaka', 'APAC'),
            (4, '567-89-0123', 'Bob Johnson', 'US'),
            (5, '678-90-1234', 'Emma Brown', 'EU')
    """)

print("\n" + "="*60)
print("✓ Setup Complete!")
print("="*60)
print("→ All tables created and populated")
print("→ Ready for demonstrations")

---

## 4. Data Masking

**What is Data Masking?**
Replaces sensitive values with obfuscated versions while maintaining format and structure.

**Use Cases:**
- Display masked SSNs (XXX-XX-6789) instead of raw values
- Preserve formats for analytics while hiding true values
- Automate masking by user/group with Unity Catalog

**Key Functions:** `IS_ACCOUNT_GROUP_MEMBER()`, `IS_MEMBER()`, `mask()`

**Important:** Fine-grained controls require serverless compute

In [0]:
# Create masking function based on group membership
spark.sql(f"""CREATE OR REPLACE {get_table_type()} FUNCTION {func_name('mask_ssn')}(ssn STRING)
RETURNS STRING
RETURN CASE
    WHEN IS_ACCOUNT_GROUP_MEMBER('admin') THEN ssn
    ELSE '***-**-****'
END""")

# Create view with masked SSN
spark.sql(f"""CREATE OR REPLACE {get_table_type()} VIEW {view_name(RETAIL_SCHEMA, 'v_customers_masked')} AS
SELECT id, {func_name('mask_ssn')}(ssn) AS ssn, name, region
FROM {table_name(RETAIL_SCHEMA, 'customers')}""")

print("Original Data:")
display(spark.sql(f"SELECT * FROM {table_name(RETAIL_SCHEMA, 'customers')} LIMIT 3"))

print("\nMasked Data (SSN hidden based on permissions):")
display(spark.sql(f"SELECT * FROM {view_name(RETAIL_SCHEMA, 'v_customers_masked')} LIMIT 3"))

print("\n✓ SSN masked based on group membership")
print("✓ Admins see full SSN, others see masked")
print("✓ Format preserved for analytics")
print("\n⚠️  Update 'admin' to your admin group name")

---

## 5. Row-Level Filtering

**What is Row Filtering?**
Controls which records users can view by applying row-level conditions, enforced transparently at query time.

**Use Cases:**
- GDPR: Restrict EU data to EU employees only
- Multi-tenancy: Each customer sees only their data
- Financial segmentation: Business units see only their accounts
- Data sharing: Curated datasets for external partners

**Benefits:** Transparent enforcement • Combines with column masks • No data duplication

In [0]:
# Create row filter function based on region
spark.sql(f"""CREATE OR REPLACE {get_table_type()} FUNCTION {func_name('filter_by_region')}(region STRING)
RETURNS BOOLEAN
RETURN CASE
    -- Add the current user for demo purposes, in production you would use the groups as shown below.
    WHEN CURRENT_USER() = '{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}' AND region = 'US' THEN TRUE
    WHEN IS_MEMBER('Team_US') AND region = 'US' THEN TRUE
    WHEN IS_MEMBER('Team_EU') AND region = 'EU' THEN TRUE
    WHEN IS_MEMBER('Team_APAC') AND region = 'APAC' THEN TRUE
    WHEN IS_ACCOUNT_GROUP_MEMBER('admin') THEN TRUE
    ELSE FALSE
END""")

# Create view with row filtering
spark.sql(f"""CREATE OR REPLACE {get_table_type()} VIEW {view_name(RETAIL_SCHEMA, 'v_customers_filtered')} AS
SELECT id, ssn, name, region
FROM {table_name(RETAIL_SCHEMA, 'customers')}
WHERE {func_name('filter_by_region')}(region)""")

print("All Data (5 rows):")
display(spark.sql(f"SELECT * FROM {table_name(RETAIL_SCHEMA, 'customers')} ORDER BY id"))

print("\nFiltered Data (based on user's region):")
display(spark.sql(f"SELECT * FROM {view_name(RETAIL_SCHEMA, 'v_customers_filtered')} ORDER BY id"))

print("\n✓ Team_US: US records only")
print("✓ Team_EU: EU records only")
print("✓ Team_APAC: APAC records only")
print("✓ Admins: All records")
print("\n⚠️  Update Team_US, Team_EU, Team_APAC to your group names")

---

## 6. Data Tokenization

**What is Tokenization?**
Substitutes sensitive values with random tokens that map back via secure vaults. Unlike hashing, tokenization is **reversible**.

**Use Cases:**
- PCI-DSS: Replace credit cards with compliant tokens
- Testing: Provide realistic but protected test data
- Analytics: Enable analysis without exposing PII
- Fraud detection: Reversible for authorized investigation

**Production Integration:** VGS, Basis Theory, TokenEx

**Trade-offs:** ✓ Reversible • ⚠️ Requires external service • ⚠️ Performance overhead

In [0]:
# Create tokenization functions
spark.sql(f"""CREATE OR REPLACE {get_table_type()} FUNCTION {func_name('tokenize')}(value STRING)
RETURNS STRING
RETURN CONCAT('TOK-', substr(sha2(value, 256), 1, 32))""")

spark.sql(f"""CREATE OR REPLACE {get_table_type()} FUNCTION {func_name('detokenize')}(token STRING, original STRING)
RETURNS STRING
RETURN CASE
    WHEN CURRENT_USER() = '{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}' AND region = 'US' THEN original
    WHEN IS_ACCOUNT_GROUP_MEMBER('CAN_SEE')  THEN original
    ELSE token
END""")

# Create tokenized data as view or table
if USE_TEMP_TABLES:
    spark.sql(f"""CREATE OR REPLACE TEMP VIEW customers_tokenized AS
    SELECT id, {func_name('tokenize')}(ssn) AS ssn_token, ssn AS ssn_original, name, region
    FROM {table_name(RETAIL_SCHEMA, 'customers')}""")
else:
    spark.sql(f"""CREATE OR REPLACE {get_table_type()} TABLE {table_name(RETAIL_SCHEMA, 'customers_tokenized')} AS
    SELECT id, {func_name('tokenize')}(ssn) AS ssn_token, ssn AS ssn_original, name, region
    FROM {table_name(RETAIL_SCHEMA, 'customers')}""")

# Create view with conditional detokenization
if USE_TEMP_TABLES:
    spark.sql(f"""CREATE OR REPLACE {get_table_type()} VIEW {view_name(RETAIL_SCHEMA, 'v_customers_tokenized')} AS
    SELECT id, {func_name('detokenize')}(ssn_token, ssn_original) AS ssn, name, region
    FROM customers_tokenized""")
else:
    spark.sql(f"""CREATE OR REPLACE {get_table_type()} VIEW {view_name(RETAIL_SCHEMA, 'v_customers_tokenized')} AS
    SELECT id, {func_name('detokenize')}(ssn_token, ssn_original) AS ssn, name, region
    FROM {table_name(RETAIL_SCHEMA, 'customers_tokenized')}""")

print("Tokenized Storage:")
display(spark.sql(f"SELECT id, ssn_token, name, region FROM {table_name(RETAIL_SCHEMA, 'customers_tokenized')} LIMIT 3"))

print("\nConditional Detokenization:")
display(spark.sql(f"SELECT * FROM {view_name(RETAIL_SCHEMA, 'v_customers_tokenized')} LIMIT 3"))

print("\n✓ Non-admins see tokens only")
print("✓ Admins see original values")
print("\n🔗 Production: VGS • Basis Theory • TokenEx")

---

## 7. Attribute-Based Access Control (ABAC)

**What is ABAC?**
Policy-driven access control based on object attributes (tags). Permissions set and enforced dynamically as data evolves.

**Use Cases:**
- Auto-deny access to columns tagged 'sensitivity=PII'
- Monitor and protect credit card data automatically
- Apply policies to new tables/columns with matching tags

**Key Features:** Tag-based policies • Dynamic enforcement • Centralized governance

**Status:** Currently in **Beta** (October 2025)

[ABAC Documentation](https://docs.databricks.com/security/attribute-based-access-control.html)

In [0]:
# ABAC Conceptual Workflow (requires Beta workspace configuration)

print("ABAC Workflow (Conceptual)")
print("="*70)

print("Step 1: Tag column as PII")
print("  ALTER TABLE hr.employee_info")
print("  ALTER COLUMN ssn SET TAGS ('sensitivity' = 'PII');\n")

print("Step 2: Create policy to mask PII")
print("  CREATE POLICY mask_pii ON SCHEMA hr")
print("  COLUMN MASK (ssn) USING '***-**-****'")
print("  TO all_accounts EXCEPT hr_admins;\n")

print("Step 3: Apply policy")
print("  ALTER TABLE hr.employee_info")
print("  ALTER COLUMN ssn SET MASK POLICY mask_pii;\n")

print("="*70)

# Simulate with view
spark.sql(f"""CREATE OR REPLACE {get_table_type()} VIEW {view_name(HR_SCHEMA, 'v_employee_info_abac')} AS
SELECT id, name, salary,
    CASE WHEN IS_ACCOUNT_GROUP_MEMBER('hr_admin') THEN ssn ELSE '***-**-****' END AS ssn
FROM {table_name(HR_SCHEMA, 'employee_info')}""")

print("Simulated ABAC Behavior:")
display(spark.sql(f"SELECT * FROM {view_name(HR_SCHEMA, 'v_employee_info_abac')}"))

print("\n✓ Policy auto-masks columns tagged as PII")
print("✓ Applies to all tables in schema")
print("✓ Exceptions for specific groups")

---

## 8. Data Encryption

**What is Encryption?**
Protects data at rest and in transit by converting to encoded format readable only with decryption keys.

**Databricks Encryption Options:**

1. **AES Functions** - Column-level encryption (`AES_ENCRYPT`, `AES_DECRYPT`)
2. **Server-side** - Automatic cloud storage encryption (S3, Azure Blob, GCS)
3. **Format-Preserving** - Encrypt while maintaining format
4. **Envelope Encryption** - Multi-layer DEK/KEK approach
5. **Multi-key Protection** - Customer + Databricks managed keys

**See `data_encryption.ipynb` for detailed encryption demonstrations**

In [0]:
# AES-128 Encryption Demo
encryption_key = "MySecureKey12345"  # ⚠️ Use Azure Key Vault/AWS KMS/GCP KMS in production

# Create encrypted data as view or table
if USE_TEMP_TABLES:
    spark.sql(f"""CREATE OR REPLACE TEMP VIEW employee_info_encrypted AS
    SELECT id, name, salary, base64(aes_encrypt(ssn, '{encryption_key}', 'ECB', 'PKCS')) AS ssn_encrypted
    FROM employee_info""")
    encrypted_table_ref = "employee_info_encrypted"
else:
    spark.sql(f"""CREATE OR REPLACE TABLE {table_name(HR_SCHEMA, 'employee_info_encrypted')} AS
    SELECT id, name, salary, base64(aes_encrypt(ssn, '{encryption_key}', 'ECB', 'PKCS')) AS ssn_encrypted
    FROM {table_name(HR_SCHEMA, 'employee_info')}""")
    encrypted_table_ref = table_name(HR_SCHEMA, 'employee_info_encrypted')

print("Encrypted Data:")
display(spark.sql(f"SELECT * FROM {encrypted_table_ref}"))

# Create view with conditional decryption
if USE_TEMP_TABLES:
    spark.sql(f"""CREATE OR REPLACE TEMP VIEW v_employee_info_decrypted AS
    SELECT id, name, salary,
        CASE
            WHEN IS_ACCOUNT_GROUP_MEMBER('hr_admin')
            THEN aes_decrypt(unbase64(ssn_encrypted), '{encryption_key}', 'ECB', 'PKCS')
            ELSE '***-**-****'
        END AS ssn
    FROM employee_info_encrypted""")
    decrypted_view_ref = "v_employee_info_decrypted"
else:
    spark.sql(f"""CREATE OR REPLACE VIEW {view_name(HR_SCHEMA, 'v_employee_info_decrypted')} AS
    SELECT id, name, salary,
        CASE
            WHEN IS_ACCOUNT_GROUP_MEMBER('hr_admin')
            THEN aes_decrypt(unbase64(ssn_encrypted), '{encryption_key}', 'ECB', 'PKCS')
            ELSE '***-**-****'
        END AS ssn
    FROM {table_name(HR_SCHEMA, 'employee_info_encrypted')}""")
    decrypted_view_ref = view_name(HR_SCHEMA, 'v_employee_info_decrypted')

print("\nConditionally Decrypted:")
display(spark.sql(f"SELECT * FROM {decrypted_view_ref}"))

print("\n✓ HR admins see decrypted values")
print("✓ Others see masked values")
print("\n🔒 Best Practices:")
print("   • Use customer-managed keys (CMK)")
print("   • Rotate keys regularly")
print("   • Store keys in vault services (Key Vault, KMS)")
print("   • Enable TLS/SSL for data in transit")

---

## Summary: Privacy Features Comparison

| Feature | Use Case | Reversible | Performance | Complexity |
|---------|----------|------------|-------------|------------|
| **RBAC** | Role-based permissions | N/A | Low | Low |
| **Views** | Controlled exposure | N/A | Low-Med | Low |
| **Hashing** | Anonymization | No | Low | Low |
| **Masking** | Format-preserving obfuscation | Optional | Low-Med | Medium |
| **Row Filtering** | Regional/attribute access | N/A | Medium | Medium |
| **Tokenization** | Reversible PII protection | Yes | Med-High | High |
| **ABAC** | Policy-driven control | N/A | Medium | Med-High |
| **Encryption** | At-rest/transit protection | Yes | Low-Med | Medium |

---

## Key Takeaways

✓ **Defense in Depth** - Combine techniques for comprehensive protection
✓ **Unity Catalog** - Centralized governance for all privacy controls
✓ **Serverless Compute** - Required for fine-grained controls
✓ **Audit & Compliance** - All controls logged and auditable
✓ **Performance** - Consider impact when implementing complex policies

---

## Resources

- [Unity Catalog](https://docs.databricks.com/data-governance/unity-catalog/index.html)
- [Row & Column Filters](https://docs.databricks.com/security/privacy/row-and-column-filters.html)
- [ABAC](https://docs.databricks.com/security/attribute-based-access-control.html)
- [Encryption](https://docs.databricks.com/security/encryption/index.html)

---

## Environment Configuration Notes

**Update these values for your Databricks environment:**

**Group Names:**
- `admin` → Your admin group
- `hr_admin` / `hr_viewer_group` → Your HR groups
- `Team_US` / `Team_EU` / `Team_APAC` → Your regional groups

**Encryption:**
- Replace hardcoded keys with Azure Key Vault / AWS KMS / GCP KMS references

**Catalogs/Schemas:**
- Adjust in Configuration cell if needed

---

## Cleanup (Only for Permanent Tables)

If you used permanent tables (`USE_TEMP_TABLES = False`), run the cleanup cell below.

**Note:** Temporary tables are automatically cleaned up when your session ends.

In [0]:
# Cleanup (only runs if using permanent tables)
if not USE_TEMP_TABLES:
    print("Cleaning up permanent tables, views, functions...")
    
    # Drop views
    for view in ['employee_info_public', 'v_employee_info_abac', 'v_employee_info_decrypted']:
        spark.sql(f"DROP VIEW IF EXISTS {CATALOG}.{HR_SCHEMA}.{view}")
    for view in ['v_customers_private']:
        spark.sql(f"DROP VIEW IF EXISTS {CATALOG}.{CUSTOMERS_SCHEMA}.{view}")
    for view in ['v_customers_masked', 'v_customers_filtered', 'v_customers_tokenized']:
        spark.sql(f"DROP VIEW IF EXISTS {CATALOG}.{RETAIL_SCHEMA}.{view}")
    
    # Drop tables
    spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{HR_SCHEMA}.employee_info")
    spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{HR_SCHEMA}.employee_info_encrypted")
    spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{CUSTOMERS_SCHEMA}.customer_info")
    spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{RETAIL_SCHEMA}.customers")
    spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{RETAIL_SCHEMA}.customers_tokenized")
    
    # Drop functions
    spark.sql(f"DROP FUNCTION IF EXISTS {CATALOG}.{GOVERNANCE_SCHEMA}.mask_ssn")
    spark.sql(f"DROP FUNCTION IF EXISTS {CATALOG}.{GOVERNANCE_SCHEMA}.filter_by_region")
    spark.sql(f"DROP FUNCTION IF EXISTS {CATALOG}.{GOVERNANCE_SCHEMA}.tokenize")
    spark.sql(f"DROP FUNCTION IF EXISTS {CATALOG}.{GOVERNANCE_SCHEMA}.detokenize")
    
    print("✓ Tables, views, and functions dropped")
    
    # Drop schemas and catalog if created by this notebook
    if CREATE_CATALOG_SCHEMA:
        print("Dropping schemas...")
        for schema in [HR_SCHEMA, CUSTOMERS_SCHEMA, RETAIL_SCHEMA, GOVERNANCE_SCHEMA]:
            spark.sql(f"DROP SCHEMA IF EXISTS {CATALOG}.{schema} CASCADE")
        print("✓ Schemas dropped")
        
        print("Dropping catalog...")
        spark.sql(f"DROP CATALOG IF EXISTS {CATALOG} CASCADE")
        print("✓ Catalog dropped")
    
    print("✓ Cleanup complete")
else:
    print("Using temporary tables - no cleanup needed!")
    print("Tables will be automatically removed when session ends.")